# DigiLab Assignment 2
Author: Emil G. Melfald, University of South-Eastern Norway <br><br>
This is the second out of two assignments related to the Digital Labs in EPE2316 Power System Analysis. The main focus in this assignment is based on the three analysis methods we have investigated so far, namely: 
- Unit Commitment (UC) 
- Economic Dispatch (ED) 
- Security Constrained Optimal Power Flow (OPF)
 
The systems you should solve will be slightly more complicated than what is shown in DigiLab 4, 5, and 6. 

# The assignment system 
The following image shows the power system you are to study during the assignment. 

![Image of the power system under study](Assignment_2_Power_System_Drawing.png)

Additional data about the components are as follows: 

#### Generators: 
| Name | P nom [MW] | p min [pu] | V set nominal [pu] | marginal cost | ramp rate [pu] | start-up cost |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| G1 | 200 | 0.01 | 1.02 | 50 | 1.0 | 500 |
| G2 | 300 | 0.8 | 1.02 | 20 | 0.3 | 500 | 
| G3 | 100 | 0.6 | 1.02 | 30 | 0.5 | 500 |

#### Lines: 
| Name | Length [km] | r [$\Omega$/km] | x [$\Omega$/km] | I max [A] | S nom [MVA] |
| ---- | ---- | ---- | ---- | ---- | ---- |
| L12 | 5 | 0.1 | 0.4 | 1000 | 228.6 | 
| L23 | 5 | 0.1 | 0.4 | 1000 | 228.6 |
| L34 | 10 | 0.2 | 0.4 | 700 | 160.0 | 
| L45 | 30 | 0.3 | 0.4 | 500 | 114.3 |
| L56 | 5 | 0.1 | 0.4 | 1000 | 228.6 |
| L61 | 5 | 0.1 | 0.4 | 1000 | 228.6 |


## Task 1: Create and simulate in PyPSA
### Task 1.1 - Create the grid 
Use the Python module PyPSA to create the network object with all the buses, lines, generators, and loads required according to the figure and tables above. 


In [2]:
import pypsa
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.optimize import root, minimize, NonlinearConstraint

V_base = 132 # kV 
S_base = 100 # MW
I_base = S_base*1e6/(np.sqrt(3)*V_base*1e3)

load_data = pd.read_csv("load_data.csv", index_col="Hour")
P_load_1_timeseries = load_data["P L1 [MW]"].values
P_load_2_timeseries = load_data["P L2 [MW]"].values
P_load_3_timeseries = load_data["P L3 [MW]"].values

Q_load_1_timeseries = load_data["Q L1 [Mvar]"].values
Q_load_2_timeseries = load_data["Q L2 [Mvar]"].values
Q_load_3_timeseries = load_data["Q L3 [Mvar]"].values

network = pypsa.Network(snapshots=range(len(P_load_1_timeseries))) 

In [3]:
# Create the network here



### Task 1.2 - Simulate the system 
Do a power flow of the system. Make three different plots, showing the following:
- Voltages at all buses over the timeseries
- Active power injection in all buses during the timeseries
- Reactive power injection in all buses during the timeseries

In [4]:
# Do the power flow here



In [5]:
# Plot the voltage magnitudes here 


In [6]:
# Plot the active power injections here


In [7]:
# Plot the reactive power injections here


### Task 1.3 - Optimize the system
Use PyPSA to do Unit Commitment for this system in the given timeframe. Answer this question by providing the following in a plot: 
- Total active load of the system
- The total spinning capacity 
- The minimum possible dispatch given the commited units

In [9]:
# Optimize the network here 


In [10]:
# Obtain the correct values and plot the data here 


### End of task 1 
--- 

## Task 2: Perform OPF

### Task 2.1: Define the objective function and constraints 
You are given a working power flow error calculation function. Your job is to define the objective function for minimizing power losses while respecting the system constraints. You should also implement the following constraints: 

**All buses**: $$ V_{min} \le V_i \le V_{max} $$

**All generators**: $$ Q_{min} \le Q_i \le Q_{max} $$ 

**All lines**: $$ 0 \le |I^{ij}| \le I_{max}^{ij} $$

The decision variables are as follows: 

$$ u = [V_1, V_3, V_5, P_3, P_5] $$

Assume the following: 

$$ V_{min} = 0.95 $$

$$ V_{max} = 1.05 $$

$$ P_{min}=0 $$ 

$$ P_{max}=P_{nom} $$ 

$$ Q_{min}=-0.3 \cdot P_{nom} $$

$$ Q_{max}=0.4 \cdot P_{nom} $$

HINT: The constraint vector may be defined in the following order: 
$$ cons = [V_2, V_4, V_6, P_1, Q_1, Q_3, Q_5, I_{12}, I_{23}, I_{34}, I_{45}, I_{56}, I_{61}] $$

In [15]:
from numeric_data import Y_bus, Y_lines

# Assumes the standard convention for y: voltage angles first, then voltages
# y = [th2, th3, th4, th5, th6, V2, V4, V6]
u0 = np.array([1.02, 1.02, 1.02, 300, 100])
th_vals = np.array([0, 0, 0, 0, 0, 0], dtype=float)
V_vals = np.array([1.02, 1.0, 1.02, 1.0, 1.02, 1.0])
P_vals = np.array([0.0, -100, 300, -120, 100, -50])/S_base 
Q_vals = np.array([0.0, -50, 0.0, -60, 0.0, -25])/S_base

#### Some helper functions

In [16]:

def get_power_flow(th_vec, V_vec): 
    V_complex = V_vec*(np.cos(th_vec) + 1j*np.sin(th_vec))
    I_inj = Y_bus @ V_complex 
    S_inj = V_complex * I_inj.conj()
    P_res = np.real(S_inj)
    Q_res = np.imag(S_inj) 
    return P_res, Q_res

def power_flow_error(y, th_vals, V_vals, P_vals, Q_vals):
    th2, th3, th4, th5, th6, V2, V4, V6 = y 
    th_vec = th_vals.copy()
    V_vec = V_vals.copy()
    th_vec[1:] = np.array([th2, th3, th4, th5, th6]) 
    V_vec[1] = V2 
    V_vec[3] = V4 
    V_vec[5] = V6
    P_res, Q_res = get_power_flow(th_vec, V_vec)

    pf_error1 = P_res[1] - P_vals[1] 
    pf_error2 = P_res[2] - P_vals[2] 
    pf_error3 = P_res[3] - P_vals[3] 
    pf_error4 = P_res[4] - P_vals[4] 
    pf_error5 = P_res[5] - P_vals[5] 
    
    pf_error6 = Q_res[1] - Q_vals[1] 
    pf_error7 = Q_res[3] - Q_vals[3] 
    pf_error8 = Q_res[5] - Q_vals[5] 
    return np.array([pf_error1, pf_error2, pf_error3, pf_error4, pf_error5, pf_error6, pf_error7, pf_error8])

def get_pf_sol(th_init, V_init, P_init, Q_init): 
    """NOTE: This returns all powers in pu. """
    th_vals = th_init.copy()
    V_vals = V_init.copy()
    P_vals = P_init.copy()
    Q_vals = Q_init.copy()

    x0 = np.array([0, 0, 0, 0, 0, 1, 1, 1], dtype=float)
    pf_sol = root(power_flow_error, x0=x0, args=(th_vals, V_vals, P_vals, Q_vals))
    th_vals[1:] = pf_sol.x[:5]
    V_vals[1]  = pf_sol.x[5]
    V_vals[3]  = pf_sol.x[6]
    V_vals[5]  = pf_sol.x[7]
    P_res, Q_res = get_power_flow(th_vals, V_vals) 
    return th_vals, V_vals, P_res, Q_res

def get_I_mat(V_vec, Y_lines):         
    V_mat = np.zeros((len(V_vec), len(V_vec)), dtype=np.complex64)
    for idx, V in enumerate(V_vec):
        V_mat[idx] = -V_vec
        V_mat[idx] += V
        V_mat[idx, idx] = V   
    I_mat = V_mat * Y_lines
    return I_mat 

In [17]:
def objective_function(u): 
    # Insert your code here 
    # HINT 1: When defining the P_vals remember that 
    # the function "get_pf_sol" needs pu values, but u gives MW values. 
    # HINT 2: Multiply the return by S_base for increased numerical stability
    return 

def opf_constraints(u): 
    # Insert your code here
    # HINT: When defining the P_vals remember that 
    # the function "get_pf_sol" needs pu values, but u gives MW values. 
    return

In [ ]:
# NOTE: All power constraints and bounds are in MW or Mvar, not pu.
# Also, current constraints are in Ampere, and voltages are in pu. 

const_min = [] # Insert values here
const_max = [] # Insert values here 
bounds = []  # Insert values here
const = NonlinearConstraint(opf_constraints, const_min, const_max)

### Task 2.2: Perform power flow and OPF 

Print out the resulting power flow solution **before and after** you have optimized $u$. The resulting power flow should have the following values. 

**Result before optimization**: 

P: [-126.59301191, -99.99999993, 300.00000034, -119.99999967, 99.99999963, -50.00000016] MW <br>
Q: [90.46918336, -49.99999898, 35.66519845, -59.99999983, 18.7217287, -25.00000125] <br>
V: [1.02, 1.01560636, 1.02, 0.99853285, 1.02, 1.01785734] pu <br>
$\theta$: [0.0, 0.01191633, 0.03352941, 0.01429365, 0.01405081, 0.00460687]

**Result from OPF**: 

P: [76.20147816, -99.99999995, 154.15605347, -119.99999993, 41.46298364, -50.00000027] MW <br>
Q: [38.30564817, -49.99999965, 78.04663076, -59.99999928, 22.46833393, -25.00000134] Mvar <br>
V: [1.04932527, 1.04553425, 1.05, 1.02835924, 1.04850317, 1.04685506] pu <br>
$\theta$: [0.0, -0.00371938, 0.00171241, -0.01312223, -0.0019218, -0.00324718] rad <br>


In [19]:
# Do the normal power flow here:


[-126.59301191  -99.99999993  300.00000034 -119.99999967   99.99999963
  -50.00000016]
[ 90.46918336 -49.99999898  35.66519845 -59.99999983  18.7217287
 -25.00000125]
[1.02       1.01560636 1.02       0.99853285 1.02       1.01785734]
[0.         0.01191633 0.03352941 0.01429365 0.01405081 0.00460687]


In [12]:
# Do the OPF here: 


### Task 2.3: Compare grid losses
What is the difference in grid losses between doing the normal power flow and doing the OPF?

#### Answer task 2.3 in this markdown cell

### End of Task 2

## End of Assignment 2
---